In [20]:
import h5py

file_path = "test_003.h5"
with h5py.File(file_path, "r") as file:
    def print_structure(name, obj):
        if isinstance(obj, h5py.Group):
            print(f"Gruppe: {name}")
        elif isinstance(obj, h5py.Dataset):
            print(f"Datensatz: {name}, Form: {obj.shape}, Datentyp: {obj.dtype}")
    
    file.visititems(print_structure)


Datensatz: augmented, Form: (3829,), Datentyp: bool
Datensatz: coords, Form: (3829, 2), Datentyp: int64
Datensatz: feats, Form: (3829, 768), Datentyp: float16


In [21]:
with h5py.File(file_path, "r") as file:
    dataset_name = "feats"
    data = file[dataset_name][:]
    print(data.shape)

(3829, 768)


In [22]:
import random
import numpy as np

mean = np.mean(data[:, :], axis=0)
std = np.std(data[:, :], axis=0)

picture_num = 1000
instance_num = 100
bad_picture_num = 500
bad_instance_num = 10
pictures = []
instances = []
for i in range(picture_num):
    for j in range(len(mean)):
        new_data = np.random.normal(loc=mean[j], scale=std[j], size=instance_num)
        instances.append(new_data)
    instances = np.array(instances).T
    instances = np.hstack((instances, np.ones((instance_num, 1)))) # labels for class 0
    instances = np.hstack((instances, np.zeros((instance_num, 1)))) # labels for class 1
    pictures.append(instances)
    instances = []

pictures = np.array(pictures, dtype=np.float32)
bad_picture_indices = random.sample(range(0, picture_num), bad_picture_num)
for i in bad_picture_indices:
    bad_instance_indices = random.sample(range(0, instance_num), bad_instance_num)
    for j in bad_instance_indices:
        pictures[i][j][0:384] = 0.0 # manipulate features
    pictures[i, :, -1] = 1.0 # switch class 1 label
    pictures[i, :, -2] = 0.0 # switch class 0 label

print(pictures.shape)
    

(1000, 100, 770)


In [ ]:
import torch
# store each picture in a file
i = 0
for picture in pictures:
    #torch.save(torch.from_numpy(picture), f"train_features/sample{i}.pt")
    torch.save(torch.from_numpy(picture), f"test_features/sample{i}.pt")
    i += 1